## Setup and Import Libraries

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
client = OpenAI()

## Creating Files

In [4]:
os.listdir('Multiple-Files/')

['.ipynb_checkpoints',
 'ACME Advertising Policy.pdf',
 'ACME_Dog_Food_Ingredients.pdf',
 'Corporate Travel Policy.pdf']

In [5]:
files_to_upload = [
    'ACME Advertising Policy.pdf',
    'ACME_Dog_Food_Ingredients.pdf',
    'Corporate Travel Policy.pdf'
]

In [6]:
def upload_assistant_files(filename):
    file = client.files.create(
        file=open(file='Multiple-Files/'+filename, mode='rb'),
        purpose='assistants'
    )
    print(file.id)

In [7]:
for file in files_to_upload:
    upload_assistant_files(file)

file-ED8nYba26PjDSKoXcoJvrA
file-7CEJpFtYEpy1W9yc1EBhoa
file-GUQoBpCXSQyKLPhA7KZWGD


In [8]:
for file in client.files.list():
    print(file.filename)
    print(file.id)
    print('\n')

Corporate Travel Policy.pdf
file-GUQoBpCXSQyKLPhA7KZWGD


ACME_Dog_Food_Ingredients.pdf
file-7CEJpFtYEpy1W9yc1EBhoa


ACME Advertising Policy.pdf
file-ED8nYba26PjDSKoXcoJvrA




## Creating Assistant

In [9]:
assistant_instructions = """You answer information about the company based on the PDF documents available in your knowledge base
"""

In [10]:
assistant = client.beta.assistants.create(
    name='Stock Visualizer',
    instructions=assistant_instructions,
    model='gpt-3.5-turbo',
    tools=[{'type':'file_search'}],
)

## Creating Thread and Message

In [11]:
thread = client.beta.threads.create()

In [12]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='Does our dog food allow for artificial ingredients?',
    attachments=[
        {"file_id": "file-ED8nYba26PjDSKoXcoJvrA", "tools": [{"type": "file_search"}]},
        {"file_id": "file-7CEJpFtYEpy1W9yc1EBhoa", "tools": [{"type": "file_search"}]},
        {"file_id": "file-ED8nYba26PjDSKoXcoJvrA", "tools": [{"type": "file_search"}]}
    ]
)

## Create Run

In [13]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [14]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

In [15]:
run.status

'completed'

## Display Response

In [16]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order='asc'
)

In [35]:
for message in messages:
    print(message)
    print("\n")

Message(id='msg_6VsFq6JtaSbmOMLwdJrpMUZk', assistant_id=None, attachments=[Attachment(file_id='file-ED8nYba26PjDSKoXcoJvrA', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')]), Attachment(file_id='file-7CEJpFtYEpy1W9yc1EBhoa', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')]), Attachment(file_id='file-ED8nYba26PjDSKoXcoJvrA', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Does our dog food allow for artificial ingredients?'), type='text')], created_at=1742251763, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_P0udh9kXQDIbVFI5avlsQVD3')


Message(id='msg_NuVxj4g8kM0u2OAR384Y19q7', assistant_id='asst_f3O4t51jdpI0p7kirc8sxrSW', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_ind

## Passing a File ID to Run

In [36]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='Use the file with file-id file-7CEJpFtYEpy1W9yc1EBhoa to answer'
)

In [37]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

In [38]:
run.status

'completed'

In [39]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order='asc'
)

In [40]:
for message in messages:
    print(message)
    print("\n")

Message(id='msg_6VsFq6JtaSbmOMLwdJrpMUZk', assistant_id=None, attachments=[Attachment(file_id='file-ED8nYba26PjDSKoXcoJvrA', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')]), Attachment(file_id='file-7CEJpFtYEpy1W9yc1EBhoa', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')]), Attachment(file_id='file-ED8nYba26PjDSKoXcoJvrA', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Does our dog food allow for artificial ingredients?'), type='text')], created_at=1742251763, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_P0udh9kXQDIbVFI5avlsQVD3')


Message(id='msg_NuVxj4g8kM0u2OAR384Y19q7', assistant_id='asst_f3O4t51jdpI0p7kirc8sxrSW', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_ind

## Delete Assistant and Files

In [41]:
response = client.beta.assistants.delete(
    assistant_id=assistant.id
)

In [42]:
response

AssistantDeleted(id='asst_f3O4t51jdpI0p7kirc8sxrSW', deleted=True, object='assistant.deleted')

In [48]:
files = client.files.list()

In [49]:
files

SyncCursorPage[FileObject](data=[FileObject(id='file-7CEJpFtYEpy1W9yc1EBhoa', bytes=35798, created_at=1742251730, filename='ACME_Dog_Food_Ingredients.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None), FileObject(id='file-ED8nYba26PjDSKoXcoJvrA', bytes=37853, created_at=1742251729, filename='ACME Advertising Policy.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)], has_more=False, object='list', first_id='file-7CEJpFtYEpy1W9yc1EBhoa', last_id='file-ED8nYba26PjDSKoXcoJvrA')

In [50]:
for file in files.data:
    print(f"File ID: {file.id} | Filename: {file.filename}")
    response = client.files.delete(file.id)
    print(f"Deleted: {file.id}, Response: {response}")

File ID: file-7CEJpFtYEpy1W9yc1EBhoa | Filename: ACME_Dog_Food_Ingredients.pdf
Deleted: file-7CEJpFtYEpy1W9yc1EBhoa, Response: FileDeleted(id='file-7CEJpFtYEpy1W9yc1EBhoa', deleted=True, object='file')
File ID: file-ED8nYba26PjDSKoXcoJvrA | Filename: ACME Advertising Policy.pdf
Deleted: file-ED8nYba26PjDSKoXcoJvrA, Response: FileDeleted(id='file-ED8nYba26PjDSKoXcoJvrA', deleted=True, object='file')
